In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import pandas as pd
from scipy.special import expit

## **Learning the Model with Synthetic Data**

Just like in the linear regression practice notebook, I'm going to generate some fake data. My X matrix will have two columns $x_1$ and $x_2$. I'm not adding a column of all ones to my data matrix this time, so when I train the model I need to remember to set ``fit_intercept = True``. Once again I will sample $x_1$ and $x_2$ from the standard normal distribution. Then I will define the log odds as $3x_1 - x_2 -1 + \epsilon$ where $\epsilon$ is the normally distributed error that linear models are assumed to have. Once I have the log odds defined, I use the sigmoid activation function to convert them to probabilities, then I use those probabilities to sample Bernoulli random variables (coin flips). The resulting Bernoulli samples will be my binary $y$ target variable.

In [ ]:
#define the practice data
np.random.seed(42)
x1 = np.random.normal(size = 5000)
x2 = np.random.normal(size = 5000)
X = np.vstack((x1, x2)).T

logits = 3 * x1 - x2 - 1 + np.random.normal(scale = 0.5, size = 5000)

#the expit function from scipy.special is the sigmoid function
probits = expit(logits)

y = np.random.binomial(n = 1, p = probits)

Now we're going to go out of order here since this is just a practice notebook and fit our logistic regression model first before testing the linearity and multicollinearity assumptions. In reality, you should always check those assumptions first. Nevertheless, let's learn a logistic regression model and see how close we get to the true coefficients of $\beta_0, \beta_1, \beta_2 = -1, 3, -1$. Remember in this context, $\beta_0$ is the intercept of the line.

In [ ]:
#use sklearn to run logistic regression
lr = LogisticRegression(fit_intercept = True, random_state = 24).fit(X, y)

#we get the intercept beta_0 using lr.intercept_, and the coefficients beta_1 and beta_2 using lr.coef_
lr.intercept_, lr.coef_

Let's plot the learned log odds versus the true log odds. Normally we wouldn't be able to do this because there is no way to observe the true log odds given just binary target variables, but since we generated the data we get to see this plot! Similarly, we can plot the true probabilities, which we normally wouldn't know, with our model's outputted probabilities.

In [ ]:
plt.plot(lr.intercept_ + X @ lr.coef_[0], logits, 'k.')
plt.plot(lr.intercept_ + X @ lr.coef_[0] ,lr.intercept_ + X @ lr.coef_[0], 'r:')
plt.xlabel('Predicted Log Odds')
plt.ylabel('True Log Odds')
plt.title('Log Odds Plot')
plt.show()

plt.plot(lr.predict_proba(X)[:,1], probits, 'k.')
plt.plot(lr.predict_proba(X)[:,1], lr.predict_proba(X)[:,1], 'r:')
plt.xlabel('Predicted Probabilities')
plt.ylabel('True Probabilities')
plt.title('Probabilities Plot')
plt.show()

#finally, print the model accuracy
acc = lr.score(X, y)
print('Accuracy: ' + str(acc))

## **Testing the Linearity in Log Odds Assumption**

Remember that Dr. Heaton taught me that since we can't actually observe the true log odds in our data, we test linearity in log odds by fitting a lowess curve to our binary data and then checking that we have monotonicity. He taught me how to do that in R using the scatter.smooth function. In the following cell I show how to do it in python.

In [ ]:
#create a lowess curve to compare x1 with y
sns.regplot(x = x1, y = y, lowess = True)
plt.xlabel('x1')
plt.ylabel('y')
plt.show()

sns.regplot(x = x2, y = y, lowess = True)
plt.xlabel('x2')
plt.ylabel('y')
plt.show()

When I say that we want monotonicity in these curves, I mean that they need to either always be decreasing or always be increasing. Both of the above plots pass that test, so both myself and Dr. Heaton would say that these plots suggest that the linearity in log odds assumption is met.

In the following cell, I show an example where the monotonicity assumption would not be met.

In [ ]:
sns.regplot(x = x1**4, y = y, lowess = True)
plt.xlabel('x1^4')
plt.ylabel('y')
plt.show()

The curve changes directions in the above plot, so there is not linearity in log odds between x1^4 and y.

**Bonus**: I like to say I invented the following function, although there is a good chance that someone else thought of this same idea before I did. We can't compare our input variables to the log odds to see if they have a linear relationship because our data doesn't have the log odds in it, it just has the binary class. However, if I group data points into bins, then I can calculate the pseudoprobability $p$ of $y = 1$ in each bin, which I can then use to calculate the log odds by taking log($\frac{p}{1-p}$). So the idea of this function is to sort the data according to the inputted x values then group them into bins. Then I calculate the pseudo probability of each bin and the log odds, and then I can plot the average x value of each bin and the log odds to see if there is a linear relationship between x and the log odds. You can also plot the pseudoprobabilities along with the average x values by setting ``use_log_odds = False``. When you plot the probabilities, you hope to see an S-shaped curve because the sigmoid curve is S-shaped, so if you have true linearity in log odds, you will have an S-shape in probabilities.

Keep in mind that I have only ever encountered one other data scientist who did something similar this, so you may not want to talk about this function in future job interviews, but I do think it is better than the above lowess curves for testing linearity, so once you get your data science jobs I'd encourage you to use this function when you train logistic regression models. Note this function will also work best with lots of data where you can have lots of bins with lots of points in each bin.

In [ ]:
def plot_log_odds(x, y, groupsize = 500, use_log_odds = True, title = '', xlab = '', ylab = ''):

    #sort the data by x
    sorted_x = x[np.argsort(x)]
    sorted_y = y[np.argsort(x)]

    predicted = []
    actual = []
    numgroups = len(x) // groupsize
    for i in range(numgroups):
        if i == numgroups-1:
            rows = [j for j in range(i*groupsize, len(x))]
        else:
            rows = [j for j in range(i*groupsize, (i+1) * groupsize)]
        group_x = sorted_x[rows]
        group_y = sorted_y[rows]
        actual.append(np.mean(group_y))
        predicted.append(np.mean(group_x))
    fig = plt.figure(figsize = (3.5,3.5))
    if use_log_odds:
        actual = np.array(actual)
        plt.plot(predicted, np.log(actual / (1-actual)), 'k.')
    else:
        plt.plot(predicted, actual, 'k.')
    plt.xlabel(xlab)
    plt.title(title)
    plt.ylabel(ylab)
    plt.show()

In [ ]:
#now let's use that function to compare x1 and y
plot_log_odds(x1, y, title = 'log odds plot', xlab = 'x1', ylab = 'log odds of y')
plot_log_odds(x1, y, use_log_odds = False, title = 'probabilities plot', xlab = 'x1', ylab = 'probability of y')

These plots are what we want to see when testing for linearity. In the first plot, there is clearly a linear relationship between x1 and the log odds, and in the second plot where we plot the pseudoprobabilities we see a slight S-shape. Next let's do the same thing with x2

In [ ]:
plot_log_odds(x2, y, title = 'log odds plot', xlab = 'x2', ylab = 'log odds of y')
plot_log_odds(x2, y, use_log_odds = False, title = 'probabilities plot', xlab = 'x2', ylab = 'probability of y')

The plots for x2 also look good. I would conclude that we have linearity in log odds from these plots.

**My expectation** is that you will check linearity in log odds using the lowess method before doing logistic regression. You can also do it with my experimental method if you would like, but you have to ask yourself who do you trust more to guide you: Matthew Heaton or Will Melville??

## **Checking multicollinearity**

Just like we did in the linear regression practice notebook, we check for multicollinearity using pair plots and correlation matrices. However, since I only have two input variables this time, I'll just plot them on one scatter plot and calculate the correlation between them.

In [ ]:
plt.plot(x1, x2, 'k.')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

print('correlation: ' + str(np.corrcoef(x1, x2)[0,1]))

There is no correlation between x1 and x2, so multicollinearity is not an issue.